In [13]:
import pandas as pd
import numpy as np
from settings import Config
from mysql_db import Database
import pdcast as pdc
import s3_upload_download as s3con
import os

# NOTEBOOK DESCRIPTION: 

In the CLAIMS table (daune), a group of policies were found, which did not appear in any of the other tables. The missing information was later acquired and stored in an S3 bucket., 

Here, the newly acquired data must be:
1. Reviewed and cleaned;
2. Integrated into the existing tables.

In [14]:
# initialise the s3_connector object needed to read/write files into an S3 bucket
s3con = s3con.s3_connector()

In [15]:
# load the files
new_pol = s3con.read('left_polite.csv')
new_obj = s3con.read('left_obiecte.csv')
new_cli = s3con.read('left_clienti.csv')

ArrowInvalid: Not a Feather V1 or Arrow IPC file